# Scaling Up with the OSPoool: A Wordcount Tutorial for Submitting Many Jobs

## Introduction

This tutorial shows you how to submit multiple jobs from a single submit file. This enables you to submit your workload without having to create one submit file per task. Use this guide as an introduction to learning how to submit a list of tasks as multiple jobs.

## Our list of tasks

Imagine you want to analyze how word usage varies from book to book or author to author over a collection of books. This is a common workflow shape, where you have a list of tasks that require different input files or parameters, but the same overall analysis.

## Getting Ready

To get started, let's make sure we are in our `tutorial-wordfreq` directory by printing our working directory:

In [ ]:
cd ~/tutorial-wordfreq

In [ ]:
pwd

You should see `/home/<username>/tutorial-wordfreq`.

If you are running this on the Guest Account, your working directory will be `/home/jovyan/tutorial-wordfreq`.

Explore the contents of this directory using the `ls` command or using the file explorer on the left.

In [ ]:
ls -lh

This directory should contain a few files and folders, similar to the file explorer on the left.

```
    ├── tutorial-wordfreq
    │   ├── /err
    │   ├── /log
    │   ├── /out
    │   ├── Alice_in_Wonderland.txt
    │   ├── Dracula.txt
    │   ├── Huckleberry_Finn.txt
    │   ├── Pride_and_Prejudice.txt
    │   ├── README.md
    │   ├── wordcount.py
    │   ├── wordcount.sub
    │   ├── WordFreq_Tutorial.ipynb
```

## Submit a single task: analyze one book

### Test the command

Let's test our analysis to understand the expected behavior. We can analyze the book `Alice_in_Wonderland.txt` by running the `wordcount.py` script:


In [ ]:
./wordcount.py Alice_in_Wonderland.txt

Run the `ls` command. You should see a new file, `counts.Alice_in_Wonderland.tsv`, which has the results of this python script. This is the output we want, and the output we expect HTCondor to return to us when our job finishes. For now, remove the output:


In [ ]:
rm counts.Alice_in_Wonderland.tsv

### Create a submit file

Let's translate this analysis into a form that HTCondor understands, so it can run the analysis for us. We describe this analysis in the form of a submit file.

Two key components of our analysis are:

1. The executable (`wordcount.py`)
2. The input file(s) (`Alice_in_Wonderland.txt`)

In HTCondor's submit file syntax, we describe the command we ran with the `executable` and `arguments` options:

```plaintext
executable = wordcount.py
arguments  = Alice_in_Wonderland.txt
```

The `executable` is the script that we want to run, and the `arguments` is
everything else that follows the script when we run it, like the test above.

The input file for this job is the `Alice_in_Wonderland.txt`
text file. While we provided the name as in the `arguments`, we *also* need
to explicitly tell HTCondor to transfer the corresponding file from the Access Point (the machine you're currently on) to the Execution Point (the machine where your job runs).

We include the file name in the following submit file option:


```plaintext
transfer_input_files = Alice_in_Wonderland.txt
```

There are other submit file options that control other aspects of the job, like
where to save error and logging information, and how many resources to request per
job. This tutorial has a sample submit file (`wordcount.sub`) with these submit file options filled in for you:

```plaintext
executable = wordcount.py
arguments = Alice_in_Wonderland.txt

log = log/job.$(Cluster).$(Process).log
error = err/job.$(Cluster).$(Process).err
output = out/job.$(Cluster).$(Process).out

transfer_input_files = Alice_in_Wonderland.txt

request_cpus   = 1
request_memory = 1GB
request_disk   = 1GB

queue 1
```

Confirm the contents of the submit file by printing it out:

In [ ]:
cat wordcount.sub

We are now ready to submit our one-book analysis to the OSPool using HTCondor.

### Submit and monitor the job

After confirming the submit file, submit the job:

In [ ]:
condor_submit wordcount.sub

HTCondor will then print out a unique job ID corresponding to this submission.

Check the job's progress using `condor_q`, which will print out a table of your job'(s) status.

In [ ]:
condor_q

You can also use the command `condor_watch_q` to monitor the
queue in real time (use the keyboard shortcut `Ctrl` + `C` to exit).

Once the job finishes, you should see the same `counts.Alice_in_Wonderland.tsv` output when you enter `ls`.

## Submit a list of tasks: analyze multiple books

Now suppose you want to analyze multiple books - more than one at a time.
You could create a separate submit file for each book, and submit all of the
files manually, but you'd have a lot of file lines to modify each time
(specifically, the `arguments` and `transfer_input_files` lines from the
previous submit file).

This would be tedious! HTCondor has options that make it easy to
submit many jobs from one submit file.

### Make a list

HTCondor can loop through a list and submit one job per line of that list - this is perfect for running the *same* analysis for *different* books!

First, let's make a list of items that are different for each task we want to run. In this example, the main difference is the books we want to analyze. So, our list should contain the names of these books.

We can easily create this list by using an `ls` command and sending the output to a text file that we'll call `book.list`:

In [ ]:
ls *.txt > book.list

The `book.list` file now contains each of the `.txt` file names in the current directory.

In [ ]:
cat book.list

The output will list the books to analyze and should look something like this:
```plaintext
Alice_in_Wonderland.txt
Dracula.txt
Huckleberry_Finn.txt
Pride_and_Prejudice.txt
```

### Modify the submit file

Next, we will make changes to our submit file so that it submits a job for
each book title in our list (seen in the `book.list` file).

Create a copy of our existing submit file, which we will use for this job submission.


In [ ]:
cp wordcount.sub many-wordcount.sub

We need to tell HTCondor which list to loop over, and what variable we want to assign that value. Open the `many-wordcount.sub` file with a text editor (i.e., `vi`, `nano`) and go to the end.

```plaintext
queue book from book.list
```

This statement works like a `for` loop:

1. HTcondor looks for the text file, `book.list`
1. HTCondor looks at the first line of `book.list` and assigns that value to the variable `book`.
1. In the submit file, every occurrence of `$(book)` is replaced by the value assigned to `book`.
1. HTCondor submits one job.
1. HTCondor then reads the second line of `book.list`, and repeats steps 2-4 until it reaches the end of the list.

> The syntax `$(variablename)` represents a submit variable whose value
> will be substituted at the time of submission.

Now, let's edit the rest of the submit file. Replace the name of the book `Alice_in_Wonderland.txt` in our submit file with the variable `$(book)`.

So, the following lines in the submit file should be changed to use the variable `$(book)`:

```
arguments = $(book)

transfer_input_files = $(book)
```

### Submit and monitor the jobs

Let's submit all of our jobs.:


In [ ]:
condor_submit many-wordcount.sub

This will now submit four jobs (one for each book on our list). Once all four
have finished running, we should see four "counts" files, one for each book in the directory.

If you don't see all four "counts" files, consider investigating the log, error, and output files and see if you can identify what caused that to happen.

#### Check Your Job Resource Usage

As you scale up your analysis, it's important to monitor the resource usage of your jobs to ensure they are running efficiently. After submitting a small number of test jobs, you can check the log files generated by HTCondor to see how much CPU time, memory, and disk space your jobs are using.

In [ ]:
cat log/*

## Next Steps

Now that you've completed the Wordcount tutorial, you're ready to build on what you've learned about submitting and managing multiple jobs on the OSPool. Here are some ideas for where to go next:

🧮 **Apply These Concepts to Your Own Workflows**
* Replace the wordcount script with your own analysis program or data-processing script.
* Use a task list (like `book.list`) to handle batches of input files or parameters for your project.
* Adjust the submit file to match your resource needs (for example, increase `request_memory` or change the number of CPUs).

📦 **Use Containers for Your Software**
* Package your Python environment or research tools in an [Apptainer container](https://portal.osg-htc.org/documentation/htc_workloads/using_software/containers-singularity/) so your software runs the same everywhere.
* If you’re familiar with Docker, see [Using Docker Containers](https://portal.osg-htc.org/documentation/htc_workloads/using_software/containers-docker/) for how to convert existing Docker images for OSPool use.

📁 **Work with Larger or More Complex Data**
* Learn how to manage input and output files efficiently across many jobs with the [Data Management Guide](https://portal.osg-htc.org/documentation/htc_workloads/managing_data/overview/).
* For large datasets, explore using the [Open Science Data Federation (OSDF)](https://portal.osg-htc.org/documentation/htc_workloads/managing_data/osdf/) for faster, distributed data access.

🚀 **Go Further with Parallel Workloads**
* Try submitting other types of parameter-sweep or ensemble analyses using the same principles.
* Explore tutorials on more advanced job structures, such as [workflow managers](https://portal.osg-htc.org/documentation/htc_workloads/automated_workflows/tutorial-pegasus/) and [domain-specific examples](https://portal.osg-htc.org/documentation/software_examples/bioinformatics/tutorial-fastqc/).

🧑‍💻 **Get Help or Collaborate**
* Contact [support@osg-htc.org](mailto:support@osg-htc.org) for one-on-one help adapting these ideas to your research.
* Join OSPool virtual office hours or training sessions—see the [Get Help](https://portal.osg-htc.org/documentation/support_and_training/support/getting-help-from-RCFs/) page for details.

### Software

In this tutorial, we used a very simple python script to demonstrate how to submit multiple jobs using HTCondor. However, most research software is more complex and has many dependencies that need to be installed on the machine where the job runs.

Our recommendation for most users is to use "Apptainer" containers for deploying their software.
For instructions on how to build an Apptainer container, see our guide [Using Apptainer/Singularity Containers](https://portal.osg-htc.org/documentation/htc_workloads/using_software/containers-singularity/).
If you are familiar with Docker, or want to learn how to use Docker, see our guide [Using Docker Containers](https://portal.osg-htc.org/documentation/htc_workloads/using_software/containers-docker/).

This information can also be found in our guide [Using Software on the Open Science Pool](https://portal.osg-htc.org/documentation/htc_workloads/using_software/software-overview/).

### Data

The ecosystem for moving data to, from, and within the HTC system can be complex, especially if trying to work with large data (> gigabytes).
For guides on how data movement works on the HTC system, see our [Data Staging and Transfer to Jobs](https://portal.osg-htc.org/documentation/htc_workloads/managing_data/overview/) guides.

### GPUs

The OSPool has GPU nodes available for common use. If you would like to learn more about our GPU capacity, please visit our [GPU Guide on the OSPool Documentation Portal](https://portal.osg-htc.org/documentation/htc_workloads/specific_resource/gpu-jobs/).

## Getting Help

The OSPool Research Computing Facilitators are here to help researchers using the OSPool for their research. We provide a broad swath of research facilitation services, including:

* **Web guides**: [OSPool Guides](https://portal.osg-htc.org/documentation/) - instructions and how-tos for using the OSPool and OSDF.
* **Email support**: get help within 1-2 business days by emailing [support@osg-htc.org](mailto:support@osg-htc.org).
* **Virtual office hours**: live discussions with facilitators - see the [Email, Office Hours, and 1-1 Meetings](https://portal.osg-htc.org/documentation/support_and_training/support/getting-help-from-RCFs/) page for current schedule.
* **One-on-one meetings**: dedicated meetings to help new users, groups get started on the system; email [support@osg-htc.org](mailto:support@osg-htc.org) to request a meeting.

This information, and more, is provided in our [Get Help](https://portal.osg-htc.org/documentation/support_and_training/support/getting-help-from-RCFs/) page.